### Project imports

In [1]:
import requests
import json
from pprint import pprint
import numpy as np
from datetime import date, timedelta
import urllib3
urllib3.disable_warnings()

## Declare global variables

In [2]:
START_DATE = '2012-01-02'
END_DATE = '2017-06-30'
user = 'epita_user_4'
pwd = 'dolphin21903'
RISK_FREE = 5e-4
server_url = 'https://dolphin.jump-technology.com:3389/api/v1/'
ratio_list = [15, 17, 18, 19, 20, 21, 22, 29]
ratio_identifiers_dict = {
    'beta' : 15, 'annual_return' : 17, 'volatility' : 18, 'correlation' : 19,
    'sharpe' : 20, 'global_return' : 21,'value_at_risk' : 22, 'action_composition': 29
}
#START_DATE = date(2012, 1, 1)
#END_DATE = date(2017, 6, 30)
#period_length = (END_DATE - START_DATE).days
#print(period_length)

## Server setup

In [3]:
class Server:
    def __init__(self, server_url, user, pwd):
        self.server_url = server_url
        self.user = user
        self.pwd = pwd
        self.session = requests.Session()
        self.session.auth = (user, pwd)

    def get(self, req_str):
        data = None
        try:
            url = server_url + req_str
            response = self.session.get(url, verify=False)
            data = json.loads(response.text)
        except Exception as e:
            print('Exception in Server Get : ', str(e))
        return data
    
    def post(self, req_str, post_obj):
        """
        req_str : the request string
        post_obj : the object to post
        """
        data = None
        try:
            url = server_url + req_str
            response = self.session.post(url=url, json=post_obj, verify=False)
            data = json.loads(response.text)
        except Exception as e:
            print('Exception : ', str(e))
        return data
    
server = Server(server_url, user, pwd)

## Get asset ids

In [4]:
#assets = server.get('asset?columns=ASSET_DATABASE_ID&columns=LABEL')
# Get assets
assets = server.get('asset')
# Get asset ids
assets_ids = [asset["ASSET_DATABASE_ID"]["value"] for asset in assets]
# Create the mapping between the assets api ids and the index
ids_idx_db = {}; ids_db_idx = {}
for idx, id_asset in enumerate(assets_ids):
    ids_idx_db[idx] = id_asset
    ids_db_idx[id_asset] = idx
    
print("Number of assets = ", len(assets_ids))

Number of assets =  528


## Get asset ratios

In [6]:
def get_assets_ratio_list(assets_ids, server, ratio_list, START_DATE, END_DATE):
    obj = {
        'ratio':ratio_list,
        'asset':assets_ids,
        'bench':None,
        'startDate':START_DATE,
        'endDate':END_DATE,
        'frequency':None
    }
    return server.post('ratio/invoke', obj)

assets_ratio_list = get_assets_ratio_list(assets_ids, server, ratio_list, START_DATE, END_DATE)

## Get asset daily returns

In [14]:
#TODO: get daily return per day, if we have a missing value in one or many assets,
# fill this value with the mean of the previous and next value.
# else if we have many missed values in the same line, then remove this line.

def get_daily_return(assets_ids, start_date, end_date):
    assets_returns = {}
    min_size = 1e10
    for asset_id in assets_ids:
        returns_obj = server.get(
            ('asset/{}/quote?start_date={}&end_date={}'.format(asset_id, start_date, end_date)))
        assets_returns[asset_id] = [r for r in returns_obj]
        min_size = min(min_size, len(assets_returns[asset_id]))
        if int(asset_id) // 10 == 0:
            print(asset_id)
    return assets_returns, min_size

#daily_return, min_size = get_daily_return(assets_ids, START_DATE, END_DATE)

In [15]:
def get_return_dates(daily_returns, assets_ids, START_DATE, END_DATE):
    """Returns all the dates in the period sorted in the increasing order"""
    nb = len(assets_ids)
    dates = {START_DATE, END_DATE}
    for asset_id in assets_ids[:nb]:
        returns = daily_return[asset_id]
        for r in returns:
            dates.add(r['date'])
    return sorted(dates, key=lambda d: tuple(map(int, d.split('-'))))

sorted_dates = get_return_dates(daily_return, assets_ids, START_DATE, END_DATE)

In [16]:
def get_assets_daily_returns(daily_returns, assets_ids, sorted_dates):
    assets_returns = np.full((len(sorted_dates), len(assets_ids)), None)
    for i, asset_id in enumerate(assets_ids):
        returns = daily_return[asset_id]
        for r in returns:
            date_idx = sorted_dates.index(r['date'])
            assets_returns[date_idx,i] = r['return']
    return assets_returns

assets_returns = get_assets_daily_returns(daily_return, assets_ids, sorted_dates)

In [80]:
# Remove none values and replace them with the mean of values before and after
none_idx = np.where(assets_returns == None)

for i, j in zip(none_idx[0], none_idx[1]):
    if i in [1, len(sorted_dates) - 2] and assets_returns[i-1,j] and assets_returns[i+1,j]:
        mean_val = (assets_returns[i-1,j] + assets_returns[i+1,j]) / 2
    elif i == 0 and assets_returns[i+1,j]:
        mean_val = assets_returns[i+1,j]
    elif i == len(sorted_dates) - 2 and assets_returns[i-1,j]:
        mean_val = assets_returns[i-1,j]
    assets_returns[i,j] = mean_val
assets_returns[none_idx] = 0    
np.where(assets_returns == None)

(array([], dtype=int64), array([], dtype=int64))

In [19]:
import pandas as pd
df = pd.DataFrame(assets_returns, index=sorted_dates)
#df.isnull().any(axis=1)
# Get the row with the same value accross all columns
df[df.apply(pd.Series.nunique, axis=1) == 1]
assets_returns[:,0].shape
df

,0,1,2,3,4,5,6,7,8,9,...,518,519,520,521,522,523,524,525,526,527
2012-01-02,0,0.00337838,0.0181261,0.02308,0,0,0,0,0.00209823,-0.00384431,...,0.0349892,0,0.0403226,0,0,0,0.0228379,0,0,0
2012-01-03,0.00871421,0.00823045,0.00601805,0.0138079,0.0120269,0,0.00497748,0.037448,0.00403575,0,...,0.0254591,0.00838628,0.0232558,0.0315675,0,0.0223393,-0.00585652,0.0096225,0.0118528,0.0207385
2012-01-04,-0.0038787,-0.0019295,-0.0194417,-0.00326108,-0.0115344,-0.00433962,0.0188679,-0.00690731,0.00474317,-0.0139894,...,0.000814001,-0.00957661,-0.00968013,0.00457263,0,0.00948258,-0.0147275,-0.00439883,-0.010635,-0.0227948
2012-01-05,-0.00230088,-0.00163581,-0.0360956,0.00230947,0.00707214,-0.0183817,0.0099537,0.00650662,-0.00295294,-0.00146771,...,0.0264335,0.00814249,-0.000849979,0.0161064,-0.0044843,0.0275679,-0.0231689,-0.0147275,0.00887989,0.0040568
2012-01-06,-0.00904737,0.0019364,0.00764768,-0.00652842,0.00948034,0.003861,0.0100848,-0.0138208,-0.000604158,0.00244978,...,-0.0174326,-0.00832913,0,-0.00275672,-0.0015015,-0.0166932,0,0.0313901,-0.00648548,0.00656566
2012-01-09,-0.000179019,-0.000371664,-0.0162261,0.0224198,-0.008,-0.00384615,0.00158838,-0.02283,0.000354829,0.00293255,...,-0.0181452,0.00534487,0.00893237,0.0127851,0,0.0204123,-0.00229533,0.015942,0.0220703,-0.00702459
2012-01-10,0.00465533,0.00713861,0.0510774,0.0117202,0.0238429,0.0341699,0.00407793,0.0148045,0.00627956,0.00925926,...,0.0324435,0.00506329,0.0354132,0.0034118,0,0.0265399,0.0272239,0.0335235,0.00152068,0.0252653
2012-01-11,-0.0122973,0.000369167,-0.00531511,-0.00130792,0.00410959,-0.00634684,-0.00293321,0.00775017,0.00401447,-0.00289715,...,0.00477327,0.0176322,-0.00203583,0.00714043,0.00601504,0,-0.00335946,-0.0110421,-0.0024294,-0.00246427
2012-01-12,-0.00469145,0.00524024,0.0381679,0.00467727,-0.00306958,0.0133383,-0.00588368,-0.0031667,0.00432999,-0.0379661,...,0.0197941,0.00247525,0.0175439,-0.00033761,-0.0104634,0.00347289,0.00262172,0.0223308,0.0185693,0.0163043
2012-01-13,0.00489485,0.000146843,0.0164216,-0.0191806,-0.00889497,0.000556174,-0.0052356,-0.0201951,-0.000271885,0.00825531,...,-0.0217391,0.00395062,-0.00761828,0,-0.010574,-0.021342,0.00224131,-0.00204778,-0.0137478,-0.0150705


## Prepare data for Optimizer

In [45]:
def get_covariance_matrix(assets_ids_target, daily_return):
    nb = len(assets_ids_target)
    # to compute the covariance matrix, we need data with the same shape -> min
    daily_return_forcov = np.zeros((nb,daily_return.shape[0]))
    for idx, asset_id in enumerate(assets_ids_target):
        daily_return_forcov[idx] = daily_return[:,idx]
    return np.cov(np.array(daily_return_forcov))


assets_ids_target = assets_ids
get_covariance_matrix(assets_ids_target, assets_returns)
assets_returns.shape[0]

1435

## Global return

In [21]:
def str_to_float(str_value):
    return float(str_value.replace(',', '.'))

def get_ratio(ratio_ids, assets_ids=assets_ids, assets_ratio_list=assets_ratio_list):
    """
    Note: to know the id of the asset value in ith index, use the mapping dict
    assets_id_dict_idx[i] -> asset_id
    """
    ratio_values = []
    for ratio_id in ratio_ids:
        aux = []
        for asset_id in assets_ids:
            aux.append(str_to_float(assets_ratio_list[asset_id][str(ratio_id)]['value']))
        ratio_values.append(np.array(aux))
    return ratio_values

ratio_ids = [ratio_identifiers_dict[x] for x in ['global_return', 'sharpe', 'volatility']]
global_returns, sharpes, volatilities = [x for x in get_ratio(ratio_ids)]
global_returns.shape

(528,)

In [75]:
def get_global_returns(asset_idx_ids, global_returns):
    aux = [int(x) for x in asset_idx_ids]
    return global_returns[aux]

get_global_returns(range(len(assets_ids)), global_returns)

array([  5.09312977e-01,   3.44875442e-01,  -5.27441048e-01,
         3.63041961e+00,   4.03746300e+00,   2.19222222e+00,
         3.51985348e+00,   1.43406250e+00,   2.27101068e-01,
        -2.88015092e-01,   3.21085714e+00,  -4.67642646e-03,
        -1.45734521e-01,   1.29506849e+01,   1.03970000e+00,
         8.82411983e-01,   3.76063940e-01,   8.54879594e-01,
         8.00300000e-01,   1.06002273e+00,   2.14550673e-01,
         3.80252982e-01,   3.52163425e-01,   2.64904275e-01,
         7.47529200e-01,  -8.55721405e-01,   5.40000000e-01,
         8.89455325e+00,   2.76355748e-01,   1.43011576e+00,
         3.39286321e+00,   4.26965957e+01,   1.08821293e+00,
         1.93777778e+00,  -3.46666667e-01,   2.67576331e+00,
         2.07605150e+00,   1.73592760e+00,  -9.91629105e-02,
        -8.50374668e-02,   1.19599249e-01,   9.88084793e-01,
        -6.22070055e-02,   6.52714147e-01,   1.06321473e+00,
         1.01772380e+00,   1.47859425e-01,  -6.44716212e-01,
         2.01363636e+00,

## Id mapping

In [47]:
def get_db_ids(idx_ids):
    return [ids_idx_db[idx_id] for idx_id in idx_ids]

def get_idx_ids(db_ids):
    ids_vector = np.zeros((len(db_ids)))
    for idx, db_id in enumerate(db_ids):
        ids_vector[idx] = ids_db_idx[db_id]
    return ids_vector

## Means

In [79]:
#def get_means(assets_returns, assets_db_ids):
#    return np.array([np.mean(assets_returns[str(idx)]) for idx in assets_db_ids])

#means = get_means(assets_returns, assets_ids)
#print(means[ids_db_idx['1']])
#print(np.mean(daily_return['1']))
#global_returns[205]

## Optimization problem

In [84]:
from cvxopt import matrix, solvers

def get_best_asset_weights(asset_target_db_ids, global_returns, assets_returns):
    # Get the idx id for the db ids
    assets_target_idx_ids = get_idx_ids(asset_target_db_ids)
    global_returns_target = get_global_returns(assets_target_idx_ids, global_returns)
    covariance_matrix = get_covariance_matrix(assets_target_idx_ids, assets_returns)
    weights = optimizer(global_returns_target, covariance_matrix)
    return weights

def optimizer(global_returns, covariance_matrix, alpha=1):
    nb = covariance_matrix.shape[0]
    # Problem definition
    P = matrix(2 * covariance_matrix, tc='d')
    q = matrix(-alpha * global_returns, tc='d')
    #q = matrix(-alpha * means, tc='d')
    G = matrix(-np.identity(nb), tc='d')
    h = matrix(np.zeros(nb))
    A = matrix(np.ones((nb))).T
    b = matrix(np.ones((1)))
    #G = matrix(np.vstack((-np.identity(nb), np.identity(nb))), tc='d')
    #h = matrix(np.hstack((-np.full((nb), 0.01), np.full((nb), 0.1))), tc='d')
    solvers.options['show_progress'] = False
    sol = solvers.qp(P, q, G, h, A, b)['x']
    return np.asarray(sol).reshape((nb,))

weights = get_best_asset_weights(assets_ids, global_returns, assets_returns)
max_idx = weights.argsort()[-20:][::-1]
weights[max_idx]

array([  9.99999970e-01,   4.96689183e-09,   4.75139986e-09,
         3.07530570e-09,   2.53468615e-09,   2.51673391e-09,
         2.00340025e-09,   1.64901816e-09,   1.44994745e-09,
         1.32015712e-09,   1.25579304e-09,   1.12243002e-09,
         8.69104811e-10,   8.35059210e-10,   8.32478857e-10,
         6.99749654e-10,   5.91444281e-10,   5.70052336e-10,
         5.28390698e-10,   4.83105150e-10])

In [85]:
def evaluate_portfolio(weights, global_returns, cov):
    volatility_portfolio = np.sqrt(weights.T @ cov @ weights)
    return_portfolio = global_returns.T @ weights
    sharpe_portfolio = (return_portfolio - RISK_FREE) / volatility_portfolio
    print(volatility_portfolio, return_portfolio)
    return sharpe_portfolio

In [243]:
weights = optimizer(assets_ids, global_returns, assets_returns)
max_idx = weights.argsort()[-20:][::-1]
weights[max_idx]

best_sharpes_idx = sharpes.argsort()[-20:][::-1]
nb = len(assets_ids)
l = list(range(nb))
asset_idxs = []
idx_ids_target = [334, 200,  66,  70, 297,  83,  19, 402, 289,  61,  90, 408,  71,
        89, 427, 181,  67, 479, 401,  99]
"""
for i in range(20):
weights = optimizer(l, global_returns, assets_returns, min_size)
max_weight_asset = np.where(weights > 0.99)[0][0]
#print(len(l), max_weight_asset)
asset_idxs.append(max_weight_asset)
l.remove(max_weight_asset)
"""

'\nfor i in range(20):\nweights = optimizer(l, global_returns, assets_returns, min_size)\nmax_weight_asset = np.where(weights > 0.99)[0][0]\n#print(len(l), max_weight_asset)\nasset_idxs.append(max_weight_asset)\nl.remove(max_weight_asset)\n'